In [1]:
import polars as pl
from polars import col

In [2]:
RAW_PATH = 'chembl_human_activities.parquet'

In [3]:
df = pl.scan_parquet(RAW_PATH)

In [4]:
df.fetch(5)

/tmp/ipykernel_10846/45131725.py:1: DeprecationWarning: `LazyFrame.fetch` is deprecated; use `LazyFrame.collect` instead, in conjunction with a call to `head`.
  df.fetch(5)


activity_id,assay_id,doc_id,record_id,molregno,standard_relation,standard_value,standard_units,standard_flag,standard_type,potential_duplicate,pchembl_value,bao_endpoint,uo_units,qudt_units,toid,upper_value,standard_upper_value,src_id,type,relation,value,units,text_value,standard_text_value
i64,i64,i64,i64,i64,str,f64,str,i64,str,i64,f64,str,str,str,null,null,null,i64,str,str,f64,str,null,null
340217,4,17841,191932,167941,""">""",10000.0,"""nM""",0,"""135""",0,null,"""BAO_0000179""","""UO_0000065""","""http://www.openphacts.org/unit…",null,null,null,1,"""135""",""">""",10000.0,"""nM""",null,null
161152,5,17430,585,556,"""=""",600000.0,"""nM""",1,"""CC50""",0,null,"""BAO_0000187""","""UO_0000065""","""http://www.openphacts.org/unit…",null,null,null,1,"""CC50""","""=""",0.0006,"""M""",null,null
162404,5,17430,584,82983,"""=""",90000.0,"""nM""",1,"""CC50""",0,null,"""BAO_0000187""","""UO_0000065""","""http://www.openphacts.org/unit…",null,null,null,1,"""CC50""","""=""",0.00009,"""M""",null,null
164804,5,17430,590,1633207,"""=""",340000.0,"""nM""",1,"""CC50""",0,null,"""BAO_0000187""","""UO_0000065""","""http://www.openphacts.org/unit…",null,null,null,1,"""CC50""","""=""",0.00034,"""M""",null,null
166046,5,17430,578,1633675,"""=""",2400.0,"""nM""",1,"""CC50""",0,null,"""BAO_0000187""","""UO_0000065""","""http://www.openphacts.org/unit…",null,null,null,1,"""CC50""","""=""",0.0000024,"""M""",null,null


In [ ]:
# Filtrowanie danych

df_filtered = df.filter(
    (col("standard_type") == "IC50") &
    (col("standard_units") == "nM") &
    (col("standard_value").is_not_null()) 
)

In [8]:
df_limited = df_filtered.select(
    col("molregno"),
    col("standard_value"),
    col("pchembl_value")
)

In [9]:
# Obliczenie pIC50 z IC50 (nM) na podstawie definicji:
# pIC50 = -log10(IC50 [M]) = -log10(IC50 [nM] / 10^9) = 9 - log10(IC50 [nM])

df_aggregated = df_limited.group_by("molregno").agg(
    IC50_nM_median = col("standard_value").median(),
    n_measurements = col("molregno").count()
).with_columns(
    pIC50 = 9 - col("IC50_nM_median").log10()
)

In [11]:
df_fin = df_aggregated.collect()

# Sprawdź wynik
print("\nFinal Activity Data:")
print(df_fin.head())
print(f"Liczba unikalnych molekuł (rekordów): {df_fin.shape[0]}")


Final Activity Data:
shape: (5, 4)
┌──────────┬────────────────┬────────────────┬──────────┐
│ molregno ┆ IC50_nM_median ┆ n_measurements ┆ pIC50    │
│ ---      ┆ ---            ┆ ---            ┆ ---      │
│ i64      ┆ f64            ┆ u32            ┆ f64      │
╞══════════╪════════════════╪════════════════╪══════════╡
│ 581744   ┆ 200.0          ┆ 1              ┆ 6.69897  │
│ 166740   ┆ 25000.0        ┆ 2              ┆ 4.60206  │
│ 133448   ┆ 8.0            ┆ 1              ┆ 8.09691  │
│ 408320   ┆ 13.513         ┆ 4              ┆ 7.869248 │
│ 423409   ┆ 1510.0         ┆ 4              ┆ 5.821023 │
└──────────┴────────────────┴────────────────┴──────────┘
Liczba unikalnych molekuł (rekordów): 198710
